# Environment Setup

## Google Colab Installation

### Install Python Environment

In [17]:
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB:
    # Code specific to Google Colab
    print("Running in Google Colab")

    # Additional setup commands for Colab
    !pip install neuralforecast
    !pip install gymnasium
    !pip install QuantStats
    !pip install -U kaleido
else:
    # Code for other environments (e.g., VS Code)
    print("Running in another environment (e.g., VS Code)")

Running in another environment (e.g., VS Code)


### Install RL Libraries

In [18]:
if IN_COLAB:
    # Retrive required files
    !wget --recursive --no-parent --cut-dirs=4 -P /content https://raw.githubusercontent.com//CodeBeckZero/MADDQN/main/environments/stockenv.py
    !wget --recursive --no-parent --cut-dirs=4 -P /content https://raw.githubusercontent.com//CodeBeckZero/MADDQN/main/utilities/cleandata.py
    !wget --recursive --no-parent --cut-dirs=4 -P /content https://raw.githubusercontent.com//CodeBeckZero/MADDQN/main/utilities/data.py
    !wget --recursive --no-parent --cut-dirs=4 -P /content https://raw.githubusercontent.com//CodeBeckZero/MADDQN/main/utilities/epsilon_decay.py
    !wget --recursive --no-parent --cut-dirs=4 -P /content https://raw.githubusercontent.com//CodeBeckZero/MADDQN/main/utilities/agentperform.py
    !wget --recursive --no-parent --cut-dirs=4 -P /content https://raw.githubusercontent.com//CodeBeckZero/MADDQN/main/utilities/prob_evaluate.py
    !wget --recursive --no-parent --cut-dirs=4 -P /content https://raw.githubusercontent.com//CodeBeckZero/MADDQN/main/agents/ddqn.py
    !wget --recursive --no-parent --cut-dirs=4 -P /content https://raw.githubusercontent.com//CodeBeckZero/MADDQN/main/agents/random.py
    !wget --recursive --no-parent --cut-dirs=4 -P /content https://raw.githubusercontent.com//CodeBeckZero/MADDQN/main/agents/baseagent.py
    !wget --recursive --no-parent --cut-dirs=4 -P /content https://raw.githubusercontent.com//CodeBeckZero/MADDQN/main/rewards/stockmarket.py
    # Move all directories and files from content/raw.githubusercontent.com to content/
    !mv /content/raw.githubusercontent.com/* /content/

    # Delete the raw.githubusercontent.com directory
    !rm -rf /content/raw.githubusercontent.com

# Activate Python Libraries & Random Seed

In [19]:
import os
import sys
import json
import torch
import optuna
import random
import numpy as np
import pandas as pd
import torch.nn as nn
import utilities.agentperform as agentperform
import utilities.cleandata as cln
from optuna.visualization import plot_parallel_coordinate
from utilities.epsilon_decay import linear_decay
from utilities.data import UniStockEnvDataStruct, TimesNetProcessing, ModifyDDQNAgentState
from utilities import prob_evaluate
from agents.ddqn import DdqnAgent
from agents.random import RandomAgent
from rewards.stockmarket import future_profit, risk_reward, zero_reward
from environments.stockenv import ContinuousOHLCVEnv
from datetime import datetime
from neuralforecast.core import NeuralForecast
from neuralforecast.models import TimesNet
from neuralforecast.losses.numpy import mae, mse
import logging
from sklearn import preprocessing
import matplotlib.pyplot as plt

#
logging.getLogger("pytorch_lightning.utilities.rank_zero").addHandler(logging.NullHandler())
logging.getLogger("pytorch_lightning.accelerators.cuda").addHandler(logging.NullHandler())
os.environ['NIXTLA_ID_AS_COL'] = '1' # Prevent Warning

def set_seed(seed):
    """Set seed for reproducibility."""
    # Python random module
    random.seed(seed)

    # NumPy
    np.random.seed(seed)

    # PyTorch
    torch.manual_seed(seed)

    # If you are using CUDA
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU.
        # Additional settings to force determinism in your operations:
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False



# Check if CUDA (GPU support) is available
if torch.cuda.is_available():
    # Get the current device
    device = torch.device("cuda:0")
    print(f"GPU Name: {torch.cuda.get_device_name(device)}")
else:
    device = 'cpu'
    print("CUDA (GPU support) is not available. PyTorch is running on CPU.")


def decimal_to_text(decimal_number):
    # Remove the decimal point and convert to integer
    integer_part = int(decimal_number * 1000)
    # Convert the integer to text
    text_representation = str(integer_part)
    return text_representation

CUDA (GPU support) is not available. PyTorch is running on CPU.


# RL Environment Setup

## Parameters & CSV Locations

In [20]:
RANDOM_SEED = 42
set_seed(RANDOM_SEED)



if not IN_COLAB:
    pwd = "C:/programming/MADDQN"
    sys.path.append(pwd)

    # Output Path Location for CSV export
    export_path = pwd + "/output_data/test/"
else:
    pwd = '/content'
    export_path = pwd + "/output_data/test/"

# Input Data Location, File Name, Stock name for labels
input_url = 'https://raw.githubusercontent.com/CodeBeckZero/MADDQN/main/input_data'

stock_inputs ={'DJI':'^DJI_daily.csv',
               'NDAQ': '^IXIC_daily.csv',
               'SP500': '^SPX_daily.csv',
               'AAPL': 'AAPL_daily.csv',
               'AMZN': 'AMZN_daily.csv',
               'GOOGL': 'GOOGL_daily.csv',
               'MSFT': 'MSFT_daily.csv',
               'SINE': 'sine_wave_daily.csv',
               'FORD': 'F_daily.csv',
               'JNJ': 'JNJ_daily.csv',
               'NEE': 'NEE_daily.csv',
               'PFE': 'PFE_daily.csv',
               'TSLA': 'TSLA_daily.csv',
               'COKE': 'COKE_daily.csv',
               'PG': 'PG_daily.csv'}

# Training Inputs
trn_keys = ['DJI','NDAQ','SP500']
training_range = ('2007-01-01','2019-12-31')
trn_dt_range = [datetime.strptime(dt_str, "%Y-%m-%d") for dt_str in training_range]

# Validation Inputs
validate = False
val_keys = trn_keys
validation_range = ('2020-01-01', '2021-12-31')
val_dt_range = [datetime.strptime(dt_str, "%Y-%m-%d") for dt_str in validation_range]

# Testing Inputs
tst_keys = ['AAPL','AMZN','GOOGL','MSFT','FORD','JNJ','NEE','PFE','TSLA','COKE','PG']
testing_range = ('2020-01-01', '2023-12-31')
tst_dt_range = [datetime.strptime(dt_str, "%Y-%m-%d") for dt_str in testing_range]

# Environmental Inputs
window_size = 28 # Needs to match the size Timesnet is trained on
price_based_on = 'close'
columns = ['open','high','low','close','volume']


# Metrics Interested in
metrics = ['n_trades','n_wins', 'win_percentage','cumulative_return','sortino','max_drawdown','sharpe', 'trade_dur_avg']

# Ranking type for Metrics
aval_metrics_rank_dic = {'n_trades':'max','n_wins': 'max' ,'n_losses':'max','win_percentage':'max','cumulative_return':'max',
                 'sortino':'max','max_drawdown':'min', 'sharpe':'max', 'trade_dur_avg':'max', 'trade_dur_min':'max',
                 'trade_dur_max':'max','buy_hold':'max'}
## See agentperform.py -> results dictionary for options

# Modifying enviornment state before the
scaling_params = {'columns': columns, 'scaling_type': 'col', 'scaler_func':preprocessing.StandardScaler()}
scaling_range = ('2007-01-01','2020-12-31')
scale_dt_range = [datetime.strptime(dt_str, "%Y-%m-%d") for dt_str in scaling_range]

# Add Noise
noise_mod_params = {'add_noise': True, 'add_noise_to_cols': columns, 'noise_mean': 0, 'noise_std': 0.1}

env_mod_params = scaling_params | noise_mod_params

## RL Enviornment Generation

In [21]:
env_data = {}
env = {}


for stock, file in stock_inputs.items():
    if stock in set(trn_keys + val_keys + tst_keys):
        # Import
        df = cln.YAHOO_csv_input(file, input_url)
        data_dic = UniStockEnvDataStruct(df,columns,price_based_on,window_size)
        env_data[stock] = data_dic
        env[stock] = ContinuousOHLCVEnv(name=stock,
                                        ohlcv_data = env_data[stock]['rw_raw_env'] ,
                                        stock_price_data= env_data[stock]['rw_raw_price_env'],
                                        commission_rate=0.005)

## Generating Indices

In [22]:
trn_idx = {}
val_idx = {}
tst_idx = {}
scale_idx = {}

for stock, file in stock_inputs.items():
    if stock in set(trn_keys + val_keys + tst_keys):
        if stock in trn_keys:
            trn_idx[stock] = env_data[stock].gen_rw_idxs(trn_dt_range)
        if stock in val_keys:
            val_idx[stock] = env_data[stock].gen_rw_idxs(val_dt_range)
        if stock in tst_keys:
            tst_idx[stock] = env_data[stock].gen_rw_idxs(tst_dt_range)

        scale_idx[stock] = env_data[stock].gen_idxs(scale_dt_range)

# Workbench Setup

## Parameters

In [23]:
## Create Directory
case_name = '/test_unit_run2'
save_path_root = pwd + case_name
os.makedirs(save_path_root, exist_ok=False)

# Timesnet
## Number of price predictions in the Future by TimesNet (Required for RL agent's input layer)
n_prediction = 5
## Need to Train TimesNet Preprocessing model (processed every cycle)
train_tn_model = False
## Importing TimesNet Preprocessing model (processed every cycle)
import_tn_model = False
tn_model_path = pwd + '/gen_data/timesnet/'
## Use Imported CSVs from Preprocessing model (no processing, straight to RL agent)
import_tn_csvs = True
tn_csvs_path = pwd + '/gen_data/csvs/'
## No modifaction of environmental state before input to agent
no_tn_preprocessing = False

# Limited Exploratory Hyperparmater Discover for single RL Agent
hyperparam_discovery = True
use_best_params = True

# Traditional Training/Testing
## Train Agent(s)
train_agent = True
# Test Agent(s)
test_agent = True



In [24]:
if  not(train_tn_model ^ import_tn_model ^ import_tn_csvs ^ no_tn_preprocessing):
    raise ValueError("Only one preprocessing options can and must be selected")


## Metric Function

Function that generates metric from enviornment that will be used during validation phase of training or testing phase of model


In [25]:
def metric_function(env):
    metric = env.step_info[-1]['New Portfolio Value'] -  env.step_info[-1]['Portfolio Value']
    return metric

# TimesNet Preprocessing

## Training

### Parameters

In [26]:
if train_tn_model:

    model = TimesNet(h = n_prediction,          # Forecast horizon
                    input_size = window_size,   # Length of Batches
                    batch_size = 1,             # Number of timeseries to predict
                    #futr_exog_list = remaining_columns,
                    hidden_size = 128,          # Size of embedding for embedding and encoders,
                    dropout = 0.40,             # Dropout for embeddings
                    conv_hidden_size = 3,       # Channels for the inception block
                    top_k = 5,                  # Top num of periods from FFT considered
                    num_kernels = 13,           # number of kernels for the inception block
                    encoder_layers = 3,         # num of encoders
                    max_steps = 1000,           # of training steps
                    early_stop_patience_steps = 10, #early stoppage on validation
                    val_check_steps = 100,      # Val check every X steps,
                    windows_batch_size = 150,   # Number of windows in training epoch,
                    num_workers_loader = 7,
                    learning_rate = 0.0003,
                    random_seed = RANDOM_SEED)

### Code Execution

In [27]:
if train_tn_model:
  nf = NeuralForecast(models=[model], freq='d')
  results = {}
  for key in trn_keys:
    results[key] = nf.fit(df=env[key],val_size=0.2)

  nf.save(path= tn_model_path,
          model_index=None,
          overwrite=True,
          save_dataset=True)

## Load Model

In [28]:
if import_tn_model:
# Define the correct path
  if IN_COLAB:

    model_path = os.path.join(os.getcwd(), 'gen_data', 'timesnet')

    # Ensure the directory and file exist
    if os.path.exists(model_path):
        nf = NeuralForecast.load(path=model_path)
    else:
        raise FileNotFoundError(f"Model path {model_path} does not exist.")

## CSV Upload

In [29]:
if import_tn_csvs:
    env_mod_func_dic = {}
    if IN_COLAB:
        # Input Data Location, File Name, Stock name for labels
        csv_path = 'https://raw.githubusercontent.com/CodeBeckZero/MADDQN/main/gen_data/csvs/'

    else:
        csv_path  = tn_csvs_path

    stock_tn ={'DJI':'DJI_tn.csv',
                'NDAQ': 'NDAQ_tn.csv',
                'SP500': 'SP500_tn.csv',
                'AAPL': 'AAPL_tn.csv',
                'AMZN': 'AMZN_tn.csv',
                'GOOGL': 'GOOGL_tn.csv',
                'MSFT': 'MSFT_tn.csv',
                'FORD': 'FORD_tn.csv',
                'JNJ': 'JNJ_tn.csv',
                'NEE': 'NEE_tn.csv',
                'PFE': 'PFE_tn.csv',
                'TSLA': 'TSLA_tn.csv',
                'COKE': 'COKE_tn.csv',
                'PG': 'PG_tn.csv'}

    for stock in set(trn_keys + val_keys + tst_keys):
        import_csv_path = f'{csv_path}/{stock_tn[stock]}'
        temp_env_mod_fuc = ModifyDDQNAgentState(uni_data=env_data[stock],
                                                csv_import=import_tn_csvs,
                                                csv_path=import_csv_path,
                                                start_scale_idx=scale_idx[stock][0],
                                                finish_scale_idx=scale_idx[stock][1],
                                                **env_mod_params)
        env_mod_func_dic[stock] = temp_env_mod_fuc



## Direct
Current direct default is to take the rolling windows last entry and convert it to a list prior to input to agent. Agent's experience memory is currently list-based

In [30]:
if no_tn_preprocessing:
    env_mod_func_dic = {}
    for stock in set(trn_keys + val_keys + tst_keys):
        temp_env_mod_fuc = ModifyDDQNAgentState(uni_data=env_data[stock],
                                                columns=columns,
                                                csv_import= None,
                                                csv_path= None,
                                                scaling_type= None,
                                                scaler_func=None,
                                                device=device)
        env_mod_func_dic[stock] = temp_env_mod_fuc

# Exploratory Hyperparameterization

## Interval Setup

In [31]:
if hyperparam_discovery:
    # Training Inputs
    hyp_training_range = ('2007-01-01','2020-12-31')
    hyp_trn_dt_range = [datetime.strptime(dt_str, "%Y-%m-%d") for dt_str in hyp_training_range]

    # Validation Inputs
    hyp_validate = False
    hyp_validation_range = ('2021-01-01', '2021-12-31')
    hyp_val_dt_range = [datetime.strptime(dt_str, "%Y-%m-%d") for dt_str in hyp_validation_range]

    # Testing Inputs
    hyp_testing_range = ('2021-01-01', '2023-12-31')
    hyp_tst_dt_range = [datetime.strptime(dt_str, "%Y-%m-%d") for dt_str in hyp_testing_range]

  # Modifying enviornment state before the
    hyp_scaling_params = {'columns': columns, 'scaling_type': 'col', 'scaler_func':preprocessing.StandardScaler()}
    hyp_scaling_range = ('2007-01-01','2020-12-31')
    hyp_scale_dt_range = [datetime.strptime(dt_str, "%Y-%m-%d") for dt_str in scaling_range]
    
    # Add Noise
    hyp_noise_mod_params = {'add_noise': True, 'add_noise_to_cols': columns, 'noise_mean': 0, 'noise_std': 0.1}

    hyp_env_mod_params = hyp_scaling_params | hyp_noise_mod_params
    
    
    hyp_trn_idx = {}
    hyp_val_idx = {}
    hyp_tst_idx = {}
    hyp_scale_idx = {}

    for stock, file in stock_inputs.items():
        if stock in set(trn_keys + val_keys + tst_keys):
            if stock in trn_keys:
                hyp_trn_idx[stock] = env_data[stock].gen_rw_idxs(hyp_trn_dt_range)
            if stock in val_keys:
                hyp_val_idx[stock] = env_data[stock].gen_rw_idxs(hyp_val_dt_range)
            if stock in tst_keys:
                hyp_tst_idx[stock] = env_data[stock].gen_rw_idxs(hyp_tst_dt_range)
            
            hyp_scale_idx[stock] = env_data[stock].gen_idxs(hyp_scale_dt_range)

    display(hyp_trn_idx,hyp_val_idx,hyp_tst_idx)

{'DJI': (0, 3497), 'NDAQ': (0, 3497), 'SP500': (0, 3497)}

{'DJI': (3525, 3749), 'NDAQ': (3525, 3749), 'SP500': (3525, 3749)}

{'AAPL': (3525, 4250),
 'AMZN': (3525, 4250),
 'GOOGL': (3525, 4250),
 'MSFT': (3525, 4250),
 'FORD': (3525, 4250),
 'JNJ': (3525, 4250),
 'NEE': (3525, 4250),
 'PFE': (3525, 4250),
 'TSLA': (2647, 3372),
 'COKE': (3525, 4250),
 'PG': (3525, 4250)}

## Parameter Search & Code Execution

In [32]:
if hyperparam_discovery:

    # For Objective function, need to create agent name before to link agent with enviornment
    agent_name = 'hyp_profit_agent'
    agent_path = save_path_root + '/hyp_param_search'
    metric = 'val_tot_r'
    max_len_buf = np.round(hyp_trn_idx['DJI'][1] - hyp_trn_idx['DJI'][0] + window_size, -2) -10 # manual input, could be error here if
    print(f'Max Mem Length: {max_len_buf}')
    
    ## Activation Function Passing
    activation_functions = {
    'ELU': nn.ELU(),
    'LRELUs02': nn.LeakyReLU(negative_slope=0.2),
    'GELU': nn.GELU()}
    

    def objective(trial):
        

        # Define the hyperparameters to search over

        ## NN hyperparameters
        #sug_hidden_layers = trial.suggest_int('hidden_layers', low=1, high=3)
        sug_hidden_size = trial.suggest_int('hidden_size', low=256, high=512, step=128)
        sug_update_q_freq = trial.suggest_int('update_q_freq',low=1,high=5)
        sug_update_tgt_freq = trial.suggest_int('update_tgt_freq',low=15,high=200)
        sug_activation_function_name = trial.suggest_categorical('activation_function', list(activation_functions.keys()))
        sug_activation_function = activation_functions[sug_activation_function_name]

        """
        ## Reward Function Passing
        reward_functions = {
        'profit': future_profit(None,5),
        'risk': risk_reward(None,5),
        }
        sug_reward_function_name = trial.suggest_categorical('reward_function', list(reward_functions.keys()))
        sug_reward_function = reward_functions[sug_reward_function_name]
        """
        ## Optimizer hyperparameters
        sug_opt_lre = trial.suggest_categorical('opt_lre',[0.0001,.0001,0.01,0.1])
        sug_gamma = trial.suggest_float('gamma',low=0.90,high=0.99,step=0.01)
        ## Memory Replay hyperparameters
        sug_buffer_size = trial.suggest_int('buffer_size',low=220,high=600,step=10)
        sug_batch_size = trial.suggest_int('batch_size',low=50,high=200,step=5)

        # Define Training Parameters
        hyp_training_params = {'training_episodes': 1,
                   'epsilon_decya_func': linear_decay,
                   'initial_epsilon': 0.9,
                   'final_epsilon': 0.1,
                   'update_q_freq': sug_update_q_freq,
                   'update_tgt_freq': sug_update_tgt_freq,                   
                   'save_path': export_path,
                   'metric_func': metric_function
                   }

        hyp_earlystop_params = {'min_training_episodes': 1,
                   'early_stop': False,
                   'stop_metric': None,
                   'stop_patience': 2,
                   'stop_delta': 0.001
                    }
        
        
        # Saving Setup
        ## Current Parameter Values:
        #cur_n_fcl = trial.params['hidden_layers']
        cur_fcl_size = trial.params['hidden_size']
        cur_q_freq = trial.params['update_q_freq']
        cur_tgt_freq = trial.params['update_tgt_freq']
        cur_act_func = trial.params['activation_function']
        #cur_rwd_func = trial.params['reward_function']
        cur_lre = decimal_to_text(trial.params['opt_lre'])
        cur_buf_size = trial.params['buffer_size']
        cur_bat_size = trial.params['batch_size']

        ## Create Notation for Hyperparameter Setup
        test_name = (f'2FC{cur_fcl_size}_{cur_act_func}_' +
                    f'BT{cur_bat_size}BF{cur_buf_size}_Q{cur_q_freq}_' +
                    f'TGT{cur_tgt_freq}_LR{cur_lre}')

        ## Create Dir to save results
        test_name_path =  agent_path + '/' + test_name
        if not os.path.exists(test_name_path):
            os.makedirs(test_name_path)
            print(f"Directory '{test_name_path}' created successfully.")
        else:
            print(f"Directory '{test_name_path}' already exists.")

        # Create Agent with hyperparameters
        best_agent = DdqnAgent(name=agent_name,
                            environment=None,
                            reward_function = future_profit,
                            reward_params = {'n':5, 'lower': 15, 'upper': 90},
                            env_state_mod_func = None,
                            input_size= 16,
                            hidden_size= sug_hidden_size,
                            output_size=3,
                            activation_function = sug_activation_function,
                            num_hidden_layers = 2,
                            buffer_size= sug_buffer_size,
                            batch_size = sug_batch_size,
                            alpha = sug_opt_lre,
                            gamma = sug_gamma,
                            opt_wgt_dcy = 0.01,
                            dropout_rate = 0.20,
                            device = device)

        # Training Model
        for key, rl_env in env.items():

            if key in trn_keys:
                rl_env.add_agent(agent_name)
                rl_env.set_decision_agent(agent_name)
                
                if hyp_validate:
                    hyp_idx_params = {'start_idx': hyp_trn_idx[key][0],
                                'end_idx':hyp_trn_idx[key][1],
                                'val_start_idx': hyp_val_idx[key][0],
                                'val_end_idx': hyp_val_idx[key][1]}
                else:
                    hyp_idx_params ={'start_idx': hyp_trn_idx[key][0],
                                'end_idx':hyp_trn_idx[key][1],
                                'val_start_idx': None,
                                'val_end_idx': None}
                
                hyp_combined_params = hyp_training_params | hyp_earlystop_params | hyp_idx_params
                
                best_agent.set_env_stat_modify_func(env_mod_func_dic[key].process)
                best_agent.set_environment(rl_env)
                best_agent.train(**hyp_combined_params)
                
                # Save Agent
                file_root_name = f'{key}_TRN_{hyp_trn_idx[key][0]}-{hyp_trn_idx[key][1]}'
                best_agent.export_Q_nn(test_name_path+"/model_save")

                # Create/Export Records
                env_data_record = rl_env.get_step_data()
                agent_data_record = best_agent.get_step_data()
                training_record =  pd.concat([env_data_record, agent_data_record], axis=1, join='inner')
                training_record.to_csv(f'{test_name_path}/{file_root_name}_step_data.csv')
                episodic_training_record = best_agent.get_training_episodic_data()
                episodic_training_record.to_csv(f'{test_name_path}/{file_root_name}_epi_data.csv')
                
                
                
                rl_env.remove_agent(agent_name)

        # Test Model

        scores = []
        for key, rl_env in env.items():

            if key in tst_keys:
                rl_env.add_agent(agent_name)
                rl_env.set_decision_agent(agent_name)
                
                best_agent.set_env_stat_modify_func(env_mod_func_dic[key].process)
                best_agent.set_environment(rl_env)
                best_agent.test(start_idx = hyp_tst_idx[key][0],
                            end_idx = hyp_tst_idx[key][1],
                            metric_func= metric_function,
                            testing_episodes=1)
                rl_env.remove_agent(agent_name)

                ## Save Test Metric Result(s) into
                ddqn_tst = best_agent.get_testing_episodic_data()
                score = ddqn_tst['tot_r'].mean()
                scores.append(score)

                ## Export Test data
                a = rl_env.get_step_data()
                b = best_agent.get_step_data()
                combined_df = pd.concat([a,b],axis=1)
                tst_df_file_name  = f'TST-{key}' + test_name + '.csv'
                trn_df_save_path = test_name_path + '/' + tst_df_file_name
                combined_df.to_csv(trn_df_save_path)

                ## Generate Trading Graphic
                tst_graph_file_name = trn_df_save_path[:-4] + '.png'
                agentperform.agent_stock_performance(env[key].stock_price_data[hyp_tst_idx[key][0]:hyp_tst_idx[key][1]][:,-1,0], # Selecting all batches, last price of window, closing price
                                                    combined_df['Env Action'].to_numpy(),
                                                    key,
                                                    best_agent.get_name(),
                                                    display_graph=False,
                                                    save_graphic=True,
                                                    path_file=tst_graph_file_name)

        mean = np.mean(scores)
        return mean

    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=2)
    
    
    best_params = study.best_params
    
    # Export best Parameters
    best_param_file_name = save_path_root + '/hyp_param_search/train_params.txt'

    with open(best_param_file_name, 'w') as file:
        for key, value in best_params.items():
            file.write(f'{key}:\t{value}')
    
    fig = plot_parallel_coordinate(study)
    fig.show()
    
    # Save the figure as a PNG file
    #fig.update_layout(width=1700, height=1100)
    #fig.write_image(save_path_root + '/hyp_param_search/parallel_coordinate_plot.png')
    
    print("Best value: ", study.best_value)
    print("Best params: ", study.best_params)   

    best_params['activation_function'] = activation_functions[best_params['activation_function']] 
    display(best_params)





[I 2024-08-05 16:36:47,981] A new study created in memory with name: no-name-b5899ea9-0cdc-4636-a547-bada2b8f08ed


Max Mem Length: 3490
Directory 'C:/programming/MADDQN/test_unit_run2/hyp_param_search/2FC256_ELU_BT180BF440_Q3_TGT131_LR0' created successfully.
DJI ENV: Agent hyp_profit_agent added
DJI ENV: Agent hyp_profit_agent assigned as decision agent

hyp_profit_agent: Training Initialized on DJI[0:3497]
hyp_profit_agent: EP 1 of 1 Finished -> ΔQ1 = 0.28, ΔQ2 = 0.31 | ∑R = -8.32, μR = -0.00 σR = 2.08                                                                                                                                                         
hyp_profit_agent: Training finished on DJI[0:3497]

hyp_profit_agent: Q-Network Exported to file "C:/programming/MADDQN/test_unit_run2/hyp_param_search/2FC256_ELU_BT180BF440_Q3_TGT131_LR0/model_save"
DJI ENV: Agent hyp_profit_agent removed
NDAQ ENV: Agent hyp_profit_agent added
NDAQ ENV: Agent hyp_profit_agent assigned as decision agent

hyp_profit_agent: Training Initialized on NDAQ[0:3497]
hyp_profit_agent: EP 1 of 1 Finished -> ΔQ1 = 0.48, ΔQ2 =

[I 2024-08-05 16:41:08,494] Trial 0 finished with value: -82355.38268181817 and parameters: {'hidden_size': 256, 'update_q_freq': 3, 'update_tgt_freq': 131, 'activation_function': 'ELU', 'opt_lre': 0.0001, 'gamma': 0.96, 'buffer_size': 440, 'batch_size': 180}. Best is trial 0 with value: -82355.38268181817.


Directory 'C:/programming/MADDQN/test_unit_run2/hyp_param_search/2FC512_LRELUs02_BT120BF290_Q5_TGT68_LR0' created successfully.
DJI ENV: Agent hyp_profit_agent added
DJI ENV: Agent hyp_profit_agent assigned as decision agent

hyp_profit_agent: Training Initialized on DJI[0:3497]
hyp_profit_agent: EP 1 of 1 Finished -> ΔQ1 = 0.65, ΔQ2 = 1.14 | ∑R = -1575.87, μR = -0.45 σR = 2.48                                                                                                                                                      
hyp_profit_agent: Training finished on DJI[0:3497]

hyp_profit_agent: Q-Network Exported to file "C:/programming/MADDQN/test_unit_run2/hyp_param_search/2FC512_LRELUs02_BT120BF290_Q5_TGT68_LR0/model_save"
DJI ENV: Agent hyp_profit_agent removed
NDAQ ENV: Agent hyp_profit_agent added
NDAQ ENV: Agent hyp_profit_agent assigned as decision agent

hyp_profit_agent: Training Initialized on NDAQ[0:3497]
hyp_profit_agent: EP 1 of 1 Finished -> ΔQ1 = 0.34, ΔQ2 = 0.28 | ∑R = 

[I 2024-08-05 16:45:59,648] Trial 1 finished with value: -82302.92134090907 and parameters: {'hidden_size': 512, 'update_q_freq': 5, 'update_tgt_freq': 68, 'activation_function': 'LRELUs02', 'opt_lre': 0.0001, 'gamma': 0.96, 'buffer_size': 290, 'batch_size': 120}. Best is trial 1 with value: -82302.92134090907.


Best value:  -82302.92134090907
Best params:  {'hidden_size': 512, 'update_q_freq': 5, 'update_tgt_freq': 68, 'activation_function': 'LRELUs02', 'opt_lre': 0.0001, 'gamma': 0.96, 'buffer_size': 290, 'batch_size': 120}


{'hidden_size': 512,
 'update_q_freq': 5,
 'update_tgt_freq': 68,
 'activation_function': LeakyReLU(negative_slope=0.2),
 'opt_lre': 0.0001,
 'gamma': 0.96,
 'buffer_size': 290,
 'batch_size': 120}

# Agent Setup

## Parameters

In [33]:
# Agent Type Setup
agent_classes = {'profit_1': DdqnAgent,
                 'profit_2': DdqnAgent,
                 #'final': DdqnAgent,
                 'random':RandomAgent}

# Mul
agent_setup = {'profit_1': ['profit_1'],
                 'profit_2': ['profit_2'],
                 'random': ['random']}
                 #'final': ['profit_1', 'profit_2']}# for multi agent key is decision agent
                 #'macro': 'macro',
                 #'opt': ['profit', 'risk', 'macro']}

agent_name_list = list(agent_classes.keys())

agents_to_train = ['profit_1','profit_2']

import_agents = False

agent_configs_to_import = {'profit_1': f'c:/Programming/MADDQN/test005/profit_1/profit_1_config.json',
                           'profit_2': f'c:/Programming/MADDQN/test005/profit_2/profit_2_config.json'}

agents_to_import = {'profit_1': f'c:/Programming/MADDQN/test005/profit_1/profit_1_model',
                    'profit_2': f'c:/Programming/MADDQN/test005/profit_2/profit_2_model'}

agent_params = {
    agent_name_list[0]:{
        'name': agent_name_list[0],
        'environment': None,
        'reward_function': future_profit,
        'reward_params': {'n':5, 'lower': 15, 'upper': 90},
        'env_state_mod_func': None,  #Is Set in Training Loop
        'input_size': 16,
        'hidden_size': 256,
        'output_size':3,
        'activation_function': nn.ReLU(),
        'num_hidden_layers': 3,
        'buffer_size': 400,
        'batch_size': 100,
        'alpha': 0.005,
        'gamma':0.90,
        'opt_wgt_dcy': 0,
        'dropout_rate': 0.15,
        'device': device
    },
    agent_name_list[1]:{
        'name': agent_name_list[1],
        'environment': None,
        'reward_function': future_profit,
        'reward_params': {'n':5, 'lower': 15, 'upper': 90},
        'env_state_mod_func': None, #Is Set in Training Loop
        'input_size': 16,
        'hidden_size': 512,
        'output_size':3,
        'activation_function': nn.ELU(),
        'num_hidden_layers': 3,
        'buffer_size': 300,
        'batch_size': 100,
        'alpha': 0.001,
        'gamma':0.90,
        'opt_wgt_dcy': 1e-3,
        'dropout_rate': 0.20,
        'device': device
    },
        agent_name_list[2]:{
        'name': agent_name_list[2],
        'environment': None,
        'reward_function': zero_reward,
        'reward_params': {},
        }}

if import_agents:
    for agent, config_loc in agent_configs_to_import.items():
        with open(config_loc, 'r') as json_file:
            config_data = json.load(json_file)
        agent_params[agent].update(config_data)
        agent_params[agent].update({'device':device})   # Ensure Current Device is used for computing GPU (config)

if use_best_params:
    # Update first agent with best parameters
    filtered_new_agent_params = {key: best_params[key] for key in agent_params[agent_name_list[0]] if key in best_params}
    agent_params[agent_name_list[0]].update(filtered_new_agent_params)

    
    


## Agent Generation

In [34]:
agents_dic = {}

for agent_name, agent_class in agent_classes.items():
    selected_agent = agent_class(**agent_params[agent_name])
    agents_dic[agent_name] = selected_agent
    if agent_name in agents_to_import.keys() and import_agents:
        agents_dic[agent_name].import_Q_nn(agents_to_import[agent_name])

print(agent_params)

{'profit_1': {'name': 'profit_1', 'environment': None, 'reward_function': <function future_profit at 0x000001EB0EB07420>, 'reward_params': {'n': 5, 'lower': 15, 'upper': 90}, 'env_state_mod_func': None, 'input_size': 16, 'hidden_size': 512, 'output_size': 3, 'activation_function': LeakyReLU(negative_slope=0.2), 'num_hidden_layers': 3, 'buffer_size': 290, 'batch_size': 120, 'alpha': 0.005, 'gamma': 0.96, 'opt_wgt_dcy': 0, 'dropout_rate': 0.15, 'device': 'cpu'}, 'profit_2': {'name': 'profit_2', 'environment': None, 'reward_function': <function future_profit at 0x000001EB0EB07420>, 'reward_params': {'n': 5, 'lower': 15, 'upper': 90}, 'env_state_mod_func': None, 'input_size': 16, 'hidden_size': 512, 'output_size': 3, 'activation_function': ELU(alpha=1.0), 'num_hidden_layers': 3, 'buffer_size': 300, 'batch_size': 100, 'alpha': 0.001, 'gamma': 0.9, 'opt_wgt_dcy': 0.001, 'dropout_rate': 0.2, 'device': 'cpu'}, 'random': {'name': 'random', 'environment': None, 'reward_function': <function zero_

# Training Parameters

In [35]:
training_params = {'training_episodes': 1,
                   'epsilon_decya_func': linear_decay,
                   'initial_epsilon': 0.9,
                   'final_epsilon': 0.1,
                   'update_q_freq': 1,
                   'update_tgt_freq': 50,
                   'save_path': export_path,
                   'metric_func': metric_function
                   }

earlystop_params = {'min_training_episodes': 1,
                   'early_stop': False,
                   'stop_metric': None,
                   'stop_patience': 2,
                   'stop_delta': 0.001
                    }



In [36]:
if use_best_params:
    # Update training params with best parameters
    filtered_traing_params = {key: best_params[key] for key in training_params if key in best_params}
    training_params.update(filtered_traing_params)

In [37]:

if train_agent:
    
    # Export Training Parameters (Does nost handle imported parameters Q_value/Tgt_value)
    train_param_file_name = save_path_root + '/train_params.txt'
    with open(train_param_file_name, 'w') as file:
        for key, value in training_params.items():
            file.write(f'{key}:\t\t{value}\n')
    # Export Early Stop Paramters:
    train_param_file_name = save_path_root + '/earlystop_params.txt'
    with open(train_param_file_name, 'w') as file:
        for key, value in training_params.items():
            file.write(f'{key}:\t\t{value}\n')   
    # Agent Params
    train_param_file_name = save_path_root + '/agent_params.txt'
    with open(train_param_file_name, 'w') as file:
        for agent in agent_params:
            file.write(f'{agent}------\n')
            for key, value in agent_params[agent].items():
                file.write(f'\t{key}:\t\t{value}\n')

    # Which Agents to Train
    filtered_agents = {
        decision_agent: agents_in_setup
        for decision_agent, agents_in_setup in agent_setup.items()
        if decision_agent in agents_to_train
    }

    for decision_agent, agents_in_setup in filtered_agents.items():
        for key in trn_keys:
            rl_env = env[key]

            if validate:
                idx_params = {'start_idx': trn_idx[key][0],
                              'end_idx':trn_idx[key][1],
                              'val_start_idx': val_idx[key][0],
                              'val_end_idx': val_idx[key][1]}
            else:
                idx_params ={'start_idx': trn_idx[key][0],
                             'end_idx':trn_idx[key][1],
                             'val_start_idx': None,
                             'val_end_idx': None}

            combined_params = training_params | earlystop_params | idx_params

            # Setup agents with environment
            rl_env.add_agent(decision_agent)
            rl_env.set_decision_agent(decision_agent)   
            agents_dic[decision_agent].set_environment(rl_env)

            # Multi-Agent Setup
            if decision_agent not in agents_in_setup or len(agents_in_setup) != 1:
                subagent_list = []
                for agent in agents_in_setup:
                    subagent_list.append(agents_dic[agent])
                env_mod_func_dic[key].add_subagents(subagent_list)
            
            #Train Agent    
            agents_dic[decision_agent].set_env_stat_modify_func(env_mod_func_dic[key].process)
            agents_dic[decision_agent].train(**combined_params)
            
            # Remove Multi-Agent Setup in ENV_MOD function
            if decision_agent not in agents_in_setup or len(agents_in_setup) != 1:
                env_mod_func_dic[key].remove_subagents()

            # Save Agent
            save_agent_path = save_path_root + f'/{decision_agent}/'
            os.makedirs(save_agent_path, exist_ok=True)
            file_root_name = f'{key}_TRN_{trn_idx[key][0]}-{trn_idx[key][1]}'
            agents_dic[decision_agent].export_Q_nn(f'{save_agent_path}{decision_agent}_model')
            agents_dic[decision_agent].save_config(f'{save_agent_path}{decision_agent}_config.json')
            
            # Create/Export Records
            env_data_record = rl_env.get_step_data()
            agent_data_record = agents_dic[decision_agent].get_step_data()
            training_record =  pd.concat([env_data_record, agent_data_record], axis=1, join='inner')
            training_record.to_csv(f'{save_agent_path}{file_root_name}_step_data.csv')
            episodic_training_record = agents_dic[decision_agent].get_training_episodic_data()
            episodic_training_record.to_csv(f'{save_agent_path}{file_root_name}_epi_data.csv')

            # Remove Agent
            rl_env.remove_agent(decision_agent)
            agents_dic[decision_agent].set_environment(None)



DJI ENV: Agent profit_1 added
DJI ENV: Agent profit_1 assigned as decision agent

profit_1: Training Initialized on DJI[0:3244]
profit_1: EP 1 of 1 Finished -> ΔQ1 = 78921.99, ΔQ2 = 117190.55 | ∑R = -3233.64, μR = -1.00 σR = 4.19                                                                                                                                                     
profit_1: Training finished on DJI[0:3244]

profit_1: Q-Network Exported to file "C:/programming/MADDQN/test_unit_run2/profit_1/profit_1_model"
DJI ENV: Agent profit_1 removed
NDAQ ENV: Agent profit_1 added
NDAQ ENV: Agent profit_1 assigned as decision agent

profit_1: Training Initialized on NDAQ[0:3244]
profit_1: EP 1 of 1 Finished -> ΔQ1 = 16366349.00, ΔQ2 = 17958438.00 | ∑R = -5844.21, μR = -1.80 σR = 4.49                                                                                                                                                
profit_1: Training finished on NDAQ[0:3244]

profit_1: Q-Networ

# Agent Testing

## Parameters

In [38]:
testing_params = {DdqnAgent: {
                   'metric_func': metric_function,
                   'metric_func_arg': {},
                   'testing_episodes':1},
                  RandomAgent: {
                    'metric_func': metric_function,
                   'metric_func_arg': {},
                   'testing_episodes':5}}

## Testing

In [39]:
if test_agent:
    result_dic_struct = ['stock','agent','test_interval','test_num']
    results = {}

    for key in tst_keys:

        # Init Record[Stock]
        results[key] = {}
        test_key = f'{tst_idx[key][0]}:{tst_idx[key][1]}'
        stock_price_data = env_data[key]['rw_raw_price_env'][tst_idx[key][0]:tst_idx[key][1],-1,0]
        rl_env = env[key]

        for decision_agent, agents_in_setup in agent_setup.items():

            # Init Record[Stock][Agent]
            results[key][decision_agent] = {}

            # Init Record[Stock][Agent][test_interval]
            results[key][decision_agent][test_key] = {}   # Different Test Keys will need loop

            # Setup agents with environment
            rl_env.add_agent(decision_agent)
            rl_env.set_decision_agent(decision_agent)   
            agents_dic[decision_agent].set_environment(rl_env)

            # Enable Randomess if Agent is of class RandomAgent
            if isinstance(agents_dic[decision_agent], RandomAgent):
                new_random_seed = random.randint(1, 10**9)
                set_seed(new_random_seed)

            # Save Agent
            save_agent_path = save_path_root + f'/{decision_agent}/'
            os.makedirs(save_agent_path, exist_ok=True)
            file_root_name = f'{key}_TST_{tst_idx[key][0]}-{tst_idx[key][1]}'

            # Multi-Agent Setup
            if decision_agent not in agents_in_setup or len(agents_in_setup) != 1:
                print('here')
                subagent_list = []
                for agent in agents_in_setup:
                    subagent_list.append(agents_dic[agent])
                env_mod_func_dic[key].add_subagents(subagent_list)
            
            
            # Test Decision Agent
            params = testing_params[agent_classes[decision_agent]]
            if not isinstance(agents_dic[decision_agent], RandomAgent):
                agents_dic[decision_agent].set_env_stat_modify_func(env_mod_func_dic[key].process)
            agents_dic[decision_agent].test(start_idx=tst_idx[key][0],
                                            end_idx=tst_idx[key][1],
                                            **params)
            
            # Remove Multi-Agent Setup in ENV_MOD function
            if decision_agent not in agents_in_setup or len(agents_in_setup) != 1:
                env_mod_func_dic[key].remove_subagents()

            # Generate Testing Records
            env_data_record = rl_env.get_step_data()
            agent_data_record = agents_dic[decision_agent].get_step_data()
            test_record =  pd.concat([env_data_record, agent_data_record], axis=1, join='inner')
            test_record.to_csv(f'{save_agent_path}{file_root_name}_step_data.csv')
            episodic_testing_record = agents_dic[decision_agent].get_testing_episodic_data()
            episodic_testing_record.to_csv(f'{save_agent_path}{file_root_name}_epi_data.csv')

            trade_actions_per_test = episodic_testing_record['tst_actions']

            for idx, action_set in enumerate(trade_actions_per_test):
                file_root_name = f'{key}_TST_{tst_idx[key][0]}-{tst_idx[key][1]}_{[idx]}'
                test_metrics = agentperform.agent_stock_performance(stock_price_ts=np.array(stock_price_data),
                                                                    trade_ts=np.array(action_set),
                                                                    stock_name=key,
                                                                    agent_name=decision_agent,
                                                                    display_graph=False,
                                                                    save_graphic= True,
                                                                    path_file = f'{save_agent_path}{file_root_name}.png')
                del test_metrics['stock']
                del test_metrics['agent_name']
                results[key][decision_agent][test_key][idx] = test_metrics

            # Remove Agent
            for agent in set([decision_agent] + agents_in_setup):
                rl_env.remove_agent(agent)
                agents_dic[agent].set_environment(None)

    display(results)
    set_seed(RANDOM_SEED)

AAPL ENV: Agent profit_1 added
AAPL ENV: Agent profit_1 assigned as decision agent

profit_1: Testing Initialized on AAPL[3272:4250]
profit_1 - AAPL[3272:4250] - Testing Finished - EP - 1 of 1-> ∑R = -98342.85, μR = -100.56, σR = 644.67
profit_1: Testing Complete on AAPL[3272:4250]

AAPL ENV: Agent profit_1 removed
AAPL ENV: Agent profit_2 added
AAPL ENV: Agent profit_2 assigned as decision agent

profit_2: Testing Initialized on AAPL[3272:4250]
profit_2 - AAPL[3272:4250] - Testing Finished - EP - 1 of 1-> ∑R = -2758.76, μR = -2.82, σR = 1992.51
profit_2: Testing Complete on AAPL[3272:4250]

AAPL ENV: Agent profit_2 removed
AAPL ENV: Agent random added
AAPL ENV: Agent random assigned as decision agent
random: Testing Initialized on AAPL[3272:4250]
random: EP 5 of 5 Finished -> ∑R = -85666.82, μR = -87.59, σR = 942.13                                                                                                                                  
random: Testing Complete on AAPL[3272:425

{'AAPL': {'profit_1': {'3272:4250': {0: {'n_trades': 480,
     'n_wins': 259,
     'n_losses': 220,
     'win_percentage': 53.95833333333333,
     'cumulative_return': 1.9449771976964352,
     'sortino': 1.8871512854400208,
     'max_drawdown': -20.29911604458361,
     'sharpe': 1.2802997735642752,
     'trade_dur_avg': 1.0,
     'trade_dur_min': 1,
     'trade_dur_max': 1,
     'buy_hold': 2.4227784730913644}}},
  'profit_2': {'3272:4250': {0: {'n_trades': 61,
     'n_wins': 38,
     'n_losses': 23,
     'win_percentage': 62.295081967213115,
     'cumulative_return': 1.7911217507109252,
     'sortino': 4.094335890217441,
     'max_drawdown': -34.35545846437793,
     'sharpe': 2.606347028834001,
     'trade_dur_avg': 14.836065573770492,
     'trade_dur_min': 1,
     'trade_dur_max': 23,
     'buy_hold': 2.4227784730913644}}},
  'random': {'3272:4250': {0: {'n_trades': 238,
     'n_wins': 128,
     'n_losses': 110,
     'win_percentage': 53.78151260504202,
     'cumulative_return': 1.30

# Aggreating Test Results

In [40]:
if test_agent:
    aggerate_results = {}
    for agent in agent_name_list:
        aggerate_results[agent] = {}
        for stock in tst_keys:
            aggerate_results[agent][stock] = {}
            test_key = f'{tst_idx[stock][0]}:{tst_idx[stock][1]}'
            aggerate_results[agent][stock][test_key] = {}
            values = np.empty((0,len(metrics)))
            for test_num in range(testing_params[agent_classes[agent]]['testing_episodes']):

                values_array = [results[stock][agent][test_key][test_num][key] for key in metrics]
                current_values = np.array(values_array)
                values = np.vstack((values,current_values))

                means_for_metrics = np.mean(values, axis=0)
                std_for_metrics = np.std(values, axis=0)

            for idx,metric in enumerate(metrics):
                aggerate_results[agent][stock][test_key][metric] = (means_for_metrics[idx],std_for_metrics[idx])

    summarized_aggerate_results = {}

    for metric in metrics:
        model_list = []
        dataset_name = []
        scores = []
        for agent in aggerate_results.keys():

            model_list.append(agent)
            score_list = []
            for stock in aggerate_results[agent].keys():
                for test in aggerate_results[agent][stock].keys():
                    run_name = stock + "-" + test
                    if run_name not in dataset_name:
                        dataset_name.append(run_name)
                    score = aggerate_results[agent][stock][test][metric][0]
                    score_list.append(np.round(score,2))
            scores.append(score_list)

        score_array = np.array(scores).T

        df = pd.DataFrame(score_array,columns=model_list)
        df['dataset'] = dataset_name

        column_order = ['dataset'] + [col for col in df.columns if col != 'dataset']
        df = df[column_order]
        summarized_aggerate_results[metric] = df


        # Export Aggreate Date to CSV
        means = df[model_list].mean()
        model_means = {model: means[model] for model in model_list}
        model_means.update({'dataset': 'mean'})
        df_export = df.copy()
        df_export.loc[len(df)] = model_means
        df_export.to_csv(f'{save_path_root}/{metric}_agg_data.csv')




    display(summarized_aggerate_results)


{'n_trades':             dataset  profit_1  profit_2  random
 0    AAPL-3272:4250     480.0      61.0   244.6
 1    AMZN-3272:4250     348.0      92.0   244.8
 2   GOOGL-3272:4250       7.0      68.0   240.4
 3    MSFT-3272:4250     488.0      62.0   243.6
 4    FORD-3272:4250       0.0     142.0   241.8
 5     JNJ-3272:4250       0.0      42.0   246.4
 6     NEE-3272:4250     478.0      73.0   244.8
 7     PFE-3272:4250       0.0      50.0   243.0
 8    TSLA-2394:3372       5.0      46.0   243.8
 9    COKE-3272:4250       6.0      57.0   245.8
 10     PG-3272:4250       0.0      34.0   240.4,
 'n_wins':             dataset  profit_1  profit_2  random
 0    AAPL-3272:4250     259.0      38.0   127.8
 1    AMZN-3272:4250     179.0      47.0   127.6
 2   GOOGL-3272:4250       5.0      37.0   127.6
 3    MSFT-3272:4250     249.0      36.0   129.4
 4    FORD-3272:4250       0.0      71.0   119.8
 5     JNJ-3272:4250       0.0      23.0   126.4
 6     NEE-3272:4250     230.0      35.0   129

# Significance Testing

In [41]:
if test_agent:
    for metric in metrics:
        display(metric)
        display(summarized_aggerate_results[metric])
        test = prob_evaluate.generate_rank_array_from_dataframe(summarized_aggerate_results[metric],
                                                                model_list,equal_rank_behav="mean",
                                                                rank_order=aval_metrics_rank_dic[metric])
        display(test)
        stat, critical_f_value, reject_null_hypo, k, n, pvalue = prob_evaluate.iman_davenport_test(test,0.95,arr_order='rows')
        display(f'n models: {k}, n_datasets {n}, ImanDavenport Stat: {stat}, Critical Value: {critical_f_value}, pvalue: {pvalue}, Reject Null Hypoth: {reject_null_hypo}')

        # Create a dictionary with the output values
        result_dict = {
                    'n_models': [k],
                    'n_datasets': [n],
                    'Iman_Davenport_Stat': [stat],
                    'Critical_Value': [critical_f_value],
                    'pvalue': [pvalue],
                    'Reject_Null_Hypothesis': [reject_null_hypo]}

        result_df = pd.DataFrame(result_dict)
        result_df.to_csv(f'{save_path_root}/{metric}_iman_davenport_test.csv')

        # Nemenyi Test
        results_raw = prob_evaluate.nemenyi_test(test,0.95,model_list)
        # Formating Output
        df_export = pd.DataFrame(results_raw, columns=['agent1_agent2', 'nemenyi_stat', 'nemenyi_threshold', 'reject_null_hypo'])
        df_export[['agent1', 'agent2']] = pd.DataFrame(df_export['agent1_agent2'].tolist(), index=df_export.index)
        df_export = df_export.drop(columns=['agent1_agent2'])
        new_column_order = ['agent1', 'agent2', 'nemenyi_stat', 'nemenyi_threshold', 'reject_null_hypo']
        df_export = df_export[new_column_order]
        df_export.to_csv(f'{save_path_root}/{metric}_nemenyi_test.csv')

        display(df_export)

'n_trades'

,dataset,profit_1,profit_2,random
0,AAPL-3272:4250,480.0,61.0,244.6
1,AMZN-3272:4250,348.0,92.0,244.8
2,GOOGL-3272:4250,7.0,68.0,240.4
3,MSFT-3272:4250,488.0,62.0,243.6
4,FORD-3272:4250,0.0,142.0,241.8
5,JNJ-3272:4250,0.0,42.0,246.4
6,NEE-3272:4250,478.0,73.0,244.8
7,PFE-3272:4250,0.0,50.0,243.0
8,TSLA-2394:3372,5.0,46.0,243.8
9,COKE-3272:4250,6.0,57.0,245.8


array([[1, 1, 3, 1, 3, 3, 1, 3, 3, 3, 3],
       [3, 3, 2, 3, 2, 2, 3, 2, 2, 2, 2],
       [2, 2, 1, 2, 1, 1, 2, 1, 1, 1, 1]])

'n models: 3, n_datasets 11, ImanDavenport Stat: 4.404761904761897, Critical Value: 3.492828476735632, pvalue: 0.03460917828870062, Reject Null Hypoth: True'

,agent1,agent2,nemenyi_stat,nemenyi_threshold,reject_null_hypo
0,profit_1,profit_2,0.090909,0.999059,False
1,profit_1,random,0.909091,0.999059,False
2,profit_2,random,1.000000,0.999059,True


...

'trade_dur_avg'

,dataset,profit_1,profit_2,random
0,AAPL-3272:4250,1.0,14.84,2.00
1,AMZN-3272:4250,1.0,9.25,2.01
2,GOOGL-3272:4250,1.0,12.85,2.06
3,MSFT-3272:4250,1.0,14.32,2.02
4,FORD-3272:4250,0.0,1.47,2.04
5,JNJ-3272:4250,0.0,7.79,1.97
6,NEE-3272:4250,1.0,10.85,1.97
7,PFE-3272:4250,0.0,5.62,1.98
8,TSLA-2394:3372,194.0,18.57,1.97
9,COKE-3272:4250,1.0,13.18,2.04


array([[3, 3, 3, 3, 3, 3, 3, 3, 1, 3, 3],
       [1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1],
       [2, 2, 2, 2, 1, 2, 2, 2, 3, 2, 2]])

'n models: 3, n_datasets 11, ImanDavenport Stat: 20.249999999999968, Critical Value: 3.492828476735632, pvalue: 0.0006338892117716599, Reject Null Hypoth: True'

,agent1,agent2,nemenyi_stat,nemenyi_threshold,reject_null_hypo
0,profit_1,profit_2,1.636364,0.999059,True
1,profit_1,random,0.818182,0.999059,False
2,profit_2,random,0.818182,0.999059,False


#Move records to Google Drive for VM

In [42]:
if IN_COLAB:
  import shutil
  from google.colab import drive

  # Connect Google Drive to download records for autoclosing of VM
  drive.mount('/content/drive')

  # Define your directory path and case name
  directory_path = save_path_root
  zip_file_name = case_name.replace("/", "")
  output_filename = f'{zip_file_name}.zip'

  # Create a zip file of the directory
  shutil.make_archive(zip_file_name, 'zip', directory_path)

  # Move the zip file to Google Drive
  shutil.move(f'{zip_file_name}.zip', f'/content/drive/My Drive/{output_filename}')